In [1]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import post_patch as pp #post patch script
#post_patch_column = 'postPatch_label'
#post_patch_pipette_column = 'endPipetteR_label'

sns.set(style = "whitegrid")
sns.set_context('notebook')

In [2]:
json_df = pd.read_csv('C:/users/kumar/documents/github/personal-projects/csv/json_metadata.csv')
json_df = json_df[json_df['status'] == 'SUCCESS']

#Creates patch_duration column and outputs in minutes
json_df['patch_duration'] = pd.to_datetime(json_df['extraction.timeRetractionEnd']) - pd.to_datetime(json_df['recording.timeWholeCellStart'])
json_df['patch_duration'] = (json_df['patch_duration'].astype('timedelta64[s]'))/60

json_df['date'] = json_df['date'].str[:10] #Strip away the time
json_df['date'] = pd.to_datetime(json_df['date']) #Converting to YYYY-MM-DD

json_df = pp.postpatch_reclass(json_df) #Makes post patch column with the script

json_df.dropna(subset = ['patch_container', 'cell_depth'], inplace = True)
json_df = json_df[json_df['patch_container'].str.match('PA')]

json_df = json_df[['date',                   
                   'patch_container', #or lims_patch_container
                   'roi_major',
                   'roi_minor',
                   'patch_duration',
                   'post_patch']]

json_df.sort_values(by = ['date'], inplace = True) #Sorts date column

C:\Users\kumar\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\kumar\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [3]:
json_df.head()

,date,patch_container,roi_major,roi_minor,patch_duration,post_patch
223,2017-10-02,PAS4_171002_451_A01,VISp,layer 4,11.866667,No-low seal
242,2017-10-03,PAS4_171003_451_A01,VISp,layer 5,14.483333,Nuc-high seal
266,2017-10-03,PAS4_171003_452_A01,VISp,layer 2/3,12.500000,Nuc-high seal
294,2017-10-04,PAS4_171004_452_A01,VISp,layer 5,9.633333,Nuc-high seal
303,2017-10-04,PAS4_171004_453_A01,VISp,layer 2/3,14.350000,Nuc-high seal


In [4]:
json_df.tail()

,date,patch_container,roi_major,roi_minor,patch_duration,post_patch
6420,2019-01-11,PAS4_190111_454_A01,VISp,layer 5,18.566667,Nuc-high seal
6463,2019-01-15,PAS4_190115_451_A01,VISp,layer 5,9.600000,Nuc-high seal
6475,2019-01-15,PAS4_190115_452_A01,VISp,layer 6a,7.650000,Nuc-high seal
6476,2019-01-15,PAS4_190115_454_A01,VISp,layer 5,12.300000,Nuc-high seal
6477,2019-01-15,PAS4_190115_453_A01,VISp,layer 6a,12.383333,Nuc-high seal


In [5]:
morph_df = pd.read_csv("C:/users/kumar/documents/github/personal-projects/csv/63x.csv")
morph_df = morph_df.loc[:,['patched_cell_container', 'ims63x_go_no_go', 'ims63x_imaging_started', 'image_series_63x_qc']]

In [6]:
morph_df.tail()

,patched_cell_container,ims63x_go_no_go,ims63x_imaging_started,image_series_63x_qc
9203,P9S4_180601_405_A01,63x no go,NaN,NaN
9204,P9S4_180601_406_A01,63x go,63x imaging started,deferred
9205,PAS4_180601_453_A01,NaN,NaN,NaN
9206,P9S4_180601_408_A01,63x no go,NaN,NaN
9207,P9S4_180601_409_A01,63x no go,NaN,NaN


In [13]:
merged_df = pd.merge(left = json_df, 
                     right = morph_df, 
                     left_on = 'patch_container',
                     right_on = 'patched_cell_container',
                     how = 'inner')
merged_df.dropna(subset=['ims63x_go_no_go'], inplace = True)
merged_df.sort_values(by = ['date'], inplace = True) #Sorts date column
merged_df.drop(columns=['patched_cell_container'], inplace=True)

In [16]:
merged_df.head(200)

,date,patch_container,roi_major,roi_minor,patch_duration,post_patch,ims63x_go_no_go,ims63x_imaging_started,image_series_63x_qc
0,2017-10-02,PAS4_171002_451_A01,VISp,layer 4,11.866667,No-low seal,63x no go,NaN,NaN
1,2017-10-03,PAS4_171003_451_A01,VISp,layer 5,14.483333,Nuc-high seal,63x no go,NaN,NaN
2,2017-10-03,PAS4_171003_452_A01,VISp,layer 2/3,12.500000,Nuc-high seal,63x go,63x imaging started,image_qc_passed
3,2017-10-04,PAS4_171004_452_A01,VISp,layer 5,9.633333,Nuc-high seal,63x go,63x imaging started,ready_for_dendrite_trace
4,2017-10-04,PAS4_171004_453_A01,VISp,layer 2/3,14.350000,Nuc-high seal,63x go,63x imaging started,deferred
5,2017-10-04,PAS4_171004_451_A01,VISp,layer 2/3,13.216667,Nuc-low seal,63x no go,NaN,NaN
7,2017-10-05,PAS4_171005_453_A01,VISp,layer 2/3,8.716667,Nuc-low seal,63x no go,NaN,NaN
8,2017-10-05,PAS4_171005_451_A01,VISp,layer 4,12.066667,No-low seal,63x no go,NaN,NaN
6,2017-10-05,PAS4_171005_452_A01,VISp,layer 2/3,12.033333,Nuc-high seal,63x go,63x imaging started,deferred
9,2017-10-06,PAS4_171006_452_A01,VISp,layer 2/3,11.516667,Nuc-high seal,63x go,63x imaging started,ready_for_dendrite_trace


In [17]:
merged_df.tail(200)

,date,patch_container,roi_major,roi_minor,patch_duration,post_patch,ims63x_go_no_go,ims63x_imaging_started,image_series_63x_qc
111,2018-01-09,PAS4_180109_451_A01,VISp,layer 2/3,10.166667,Nuc-low seal,63x go,63x imaging started,image_qc_passed
113,2018-01-10,PAS4_180110_451_A01,VISp,layer 5,10.750000,Nuc-high seal,63x no go,NaN,NaN
114,2018-01-10,PAS4_180110_452_A01,VISp,layer 5,9.683333,Nuc-high seal,63x no go,NaN,NaN
116,2018-01-11,PAS4_180111_451_A01,VISp,layer 2/3,10.083333,Nuc-high seal,63x no go,NaN,NaN
117,2018-01-12,PAS4_180112_454_A01,VISp,layer 5,8.650000,Nuc-high seal,63x no go,NaN,NaN
120,2018-01-12,PAS4_180112_453_A01,VISp,layer 2/3,8.800000,Nuc-low seal,63x go,63x imaging started,image_qc_passed
125,2018-01-15,PAS4_180115_451_A01,VISp,layer 2/3,9.283333,Nuc-high seal,63x no go,NaN,NaN
126,2018-01-15,PAS4_180115_452_A01,VISp,layer 5,9.716667,Nuc-high seal,63x go,63x imaging started,deferred
127,2018-01-15,PAS4_180115_455_A01,VISp,layer 2/3,7.700000,Nuc-high seal,63x go,63x imaging started,NaN
128,2018-01-15,PAS4_180115_456_A01,VISp,layer 2/3,9.866667,Nuc-high seal,63x no go,NaN,NaN
